In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()

import numpy 
import json
import tensorflow 
import random 
import tflearn
import pickle
import zipfile
from tensorflow import keras

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [1]:
!pip install tflearn

     |████████████████████████████████| 112kB 6.0MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=13304c81da712a238167224fbbdd7532f169f462bb70406d7fe8bd19b1142dea
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
zip_ref = zipfile.ZipFile("drive/My Drive/All Project Works/Software Engineering Project/ChatterBot.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [4]:
cd 'drive/My Drive/All Project Works/Software Engineering Project'

/content/drive/My Drive/All Project Works/Software Engineering Project


In [5]:
ls

'AI Chatbot.ipynb'   data.pickle                     model.mdl.index
 Chatbot.py          intents.json                    model.mdl.meta
 ChatterBot.zip     'json file'/
 checkpoint          model.mdl.data-00000-of-00001


In [6]:
 file = "intents.json"
 
 with open("intents.json") as file:
    data=json.load(file)
    
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies ar

In [7]:
try:
  with open("data.pickle","rb") as f:
    words, labels, training, output = pickle.load(f)
except:
  words = []
  labels = []
  docs_x = []
  docs_y = []

  for intent in data["intents"]:
      for pattern in intent["patterns"]:
          wrds = nltk.word_tokenize(pattern)
          words.extend(wrds)
          docs_x.append(wrds)
          docs_y.append(intent["tag"])
          
          if intent["tag"] not in labels:
              labels.append(intent["tag"])

  words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty=[0 for _ in range (len(labels))]

  for x, doc in enumerate(docs_x):
      bag = []

      wrds = [stemmer.stem(w.lower()) for w in doc]
      
      for w in words:
          if w in wrds:
              bag.append(1)
          else:
              bag.append(0)
      
      
      output_row = out_empty[:]
      output_row[labels.index(docs_y[x])] = 1
      
      training.append(bag)
      output.append(output_row)

  
    
  training = numpy.array(training)
  output = numpy.array(output)

  with open("data.pickle","wb") as f:
    pickle.dump((words, labels, training, output),f)

tensorflow.compat.v1.reset_default_graph()

In [8]:
len(output[0])

6

In [9]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation="softmax") 
net = tflearn.regression(net)

model = tflearn.DNN(net)


model.fit(training, output, n_epoch=1000, batch_size=8,show_metric=True)
model.save("model.mdl")

Training Step: 3999  | total loss: 0.00992 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.00992 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.01016 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.01016 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/drive/My Drive/All Project Works/Software Engineering Project/model.mdl is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
def bag_of_words(sentence,words):
  bag = [0 for _ in range(len(words))]

  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]

  for se in sentence_words:
    for i,w in enumerate(words):
      if w == se:
        bag[i] = 1

  return numpy.array(bag)


In [10]:
model.load("model.mdl")

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/All Project Works/Software Engineering Project/model.mdl


In [11]:
def chat():
  print("Start talking with the bot!")

  while True:

    inp = input("You: ")

    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp,words)])

    results_index = numpy.argmax(results)

    tag = labels[results_index]

    for tg in data["intents"]:
      if tg["tag"] == tag:
        responses = tg["responses"]

    print(random.choice(responses))

In [12]:
chat()

Start talking with the bot!
You: Hello
Good to see you again!
You: quit
